In [77]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
%matplotlib inline
# ops
import warnings
warnings.filterwarnings('ignore')

seed_number = 123

In [78]:
main_dir = os.path.abspath(os.getcwd())

main_dir

'/home/pedro/Downloads/xgboost'

### ETAPA 1: lendo a base de dados

In [79]:
train = pd.read_csv(f'{main_dir}/input/train.csv')
test = pd.read_csv(f'{main_dir}/input/test.csv')
train.shape, test.shape

((113754, 24), (28439, 24))

In [80]:
data = pd.concat([train,test],sort=True)
data.columns

Index(['Cloud3pm', 'Cloud9am', 'Date', 'Evaporation', 'Humidity3pm',
       'Humidity9am', 'Id', 'Location', 'MaxTemp', 'MinTemp', 'Pressure3pm',
       'Pressure9am', 'RainToday', 'RainTomorrow', 'Rainfall', 'Sunshine',
       'Temp3pm', 'Temp9am', 'WindDir3pm', 'WindDir9am', 'WindGustDir',
       'WindGustSpeed', 'WindSpeed3pm', 'WindSpeed9am'],
      dtype='object')

In [81]:
data.shape, data.columns

((142193, 24),
 Index(['Cloud3pm', 'Cloud9am', 'Date', 'Evaporation', 'Humidity3pm',
        'Humidity9am', 'Id', 'Location', 'MaxTemp', 'MinTemp', 'Pressure3pm',
        'Pressure9am', 'RainToday', 'RainTomorrow', 'Rainfall', 'Sunshine',
        'Temp3pm', 'Temp9am', 'WindDir3pm', 'WindDir9am', 'WindGustDir',
        'WindGustSpeed', 'WindSpeed3pm', 'WindSpeed9am'],
       dtype='object'))

In [82]:
data.head()

,Cloud3pm,Cloud9am,Date,Evaporation,Humidity3pm,Humidity9am,Id,Location,MaxTemp,MinTemp,...,Rainfall,Sunshine,Temp3pm,Temp9am,WindDir3pm,WindDir9am,WindGustDir,WindGustSpeed,WindSpeed3pm,WindSpeed9am
0,NaN,NaN,2008-12-02,NaN,25.0,44.0,2,Albury,25.1,7.4,...,0.0,NaN,24.3,17.2,WSW,NNW,WNW,44.0,22.0,4.0
1,2.0,NaN,2008-12-03,NaN,30.0,38.0,3,Albury,25.7,12.9,...,0.0,NaN,23.2,21.0,WSW,W,WSW,46.0,26.0,19.0
2,NaN,NaN,2008-12-04,NaN,16.0,45.0,4,Albury,28.0,9.2,...,0.0,NaN,26.5,18.1,E,SE,NE,24.0,9.0,11.0
3,NaN,NaN,2008-12-06,NaN,23.0,55.0,6,Albury,29.7,14.6,...,0.2,NaN,28.9,20.6,W,W,WNW,56.0,24.0,19.0
4,NaN,NaN,2008-12-08,NaN,19.0,48.0,8,Albury,26.7,7.7,...,0.0,NaN,25.5,16.3,W,SSE,W,35.0,17.0,6.0


In [83]:
data.describe()

,Cloud3pm,Cloud9am,Evaporation,Humidity3pm,Humidity9am,Id,MaxTemp,MinTemp,Pressure3pm,Pressure9am,Rainfall,Sunshine,Temp3pm,Temp9am,WindGustSpeed,WindSpeed3pm,WindSpeed9am
count,85099.000000,88536.000000,81350.000000,138583.000000,140419.000000,142193.000000,141871.000000,141556.000000,128212.000000,128179.000000,140787.000000,74377.000000,139467.000000,141289.000000,132923.000000,139563.000000,140845.000000
mean,4.503167,4.437189,5.469824,51.482606,68.843810,72885.900649,23.226784,12.186400,1015.258204,1017.653758,2.349974,7.624853,21.687235,16.987509,39.984292,18.637576,14.001988
std,2.720633,2.887016,4.188537,20.797772,19.051293,42069.147355,7.117618,6.403283,7.036677,7.105476,8.465173,3.781525,6.937594,6.492838,13.588801,8.803345,8.893337
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-4.800000,-8.500000,977.100000,980.500000,0.000000,0.000000,-5.400000,-7.200000,6.000000,0.000000,0.000000
25%,2.000000,1.000000,2.600000,37.000000,57.000000,36236.000000,17.900000,7.600000,1010.400000,1012.900000,0.000000,4.900000,16.600000,12.300000,31.000000,13.000000,7.000000
50%,5.000000,5.000000,4.800000,52.000000,70.000000,73312.000000,22.600000,12.000000,1015.200000,1017.600000,0.000000,8.500000,21.100000,16.700000,39.000000,19.000000,13.000000
75%,7.000000,7.000000,7.400000,66.000000,83.000000,109248.000000,28.200000,16.800000,1020.000000,1022.400000,0.800000,10.600000,26.400000,21.600000,48.000000,24.000000,19.000000
max,9.000000,9.000000,145.000000,100.000000,100.000000,145459.000000,48.100000,33.900000,1039.600000,1041.000000,371.000000,14.500000,46.700000,40.200000,135.000000,87.000000,130.000000


In [84]:
data.describe(include=['O'])

,Date,Location,RainToday,RainTomorrow,WindDir3pm,WindDir9am,WindGustDir
count,142193,142193,140787,142193,138415,132180,132863
unique,3436,49,2,2,16,16,16
top,2017-05-10,Canberra,No,No,SE,N,W
freq,49,3418,109332,110316,10663,11393,9780


In [85]:
# checando quais atributos são numéricos (não consegui exibir o dataframe inteiro no vscode)
numerics =  data.select_dtypes(include = np.number).columns.to_list()
print(numerics)

"""
17 atributos numéricos no total
"""

['Cloud3pm', 'Cloud9am', 'Evaporation', 'Humidity3pm', 'Humidity9am', 'Id', 'MaxTemp', 'MinTemp', 'Pressure3pm', 'Pressure9am', 'Rainfall', 'Sunshine', 'Temp3pm', 'Temp9am', 'WindGustSpeed', 'WindSpeed3pm', 'WindSpeed9am']


'\n17 atributos numéricos no total\n'

In [86]:
#checando atributos categoricos
categorical = data.select_dtypes('object').columns.to_list()
print(categorical)

"""
7 atributos categóricos
"""

['Date', 'Location', 'RainToday', 'RainTomorrow', 'WindDir3pm', 'WindDir9am', 'WindGustDir']


'\n7 atributos categóricos\n'

### Etapa 2: Limpeza e preparação dos Dados

In [87]:
# o ID é inútil pq só atribui um valor unico pra a instancia do dado então vou dropar
data.drop("Id", axis=1, inplace=True)
data

,Cloud3pm,Cloud9am,Date,Evaporation,Humidity3pm,Humidity9am,Location,MaxTemp,MinTemp,Pressure3pm,...,Rainfall,Sunshine,Temp3pm,Temp9am,WindDir3pm,WindDir9am,WindGustDir,WindGustSpeed,WindSpeed3pm,WindSpeed9am
0,NaN,NaN,2008-12-02,NaN,25.0,44.0,Albury,25.1,7.4,1007.8,...,0.0,NaN,24.3,17.2,WSW,NNW,WNW,44.0,22.0,4.0
1,2.0,NaN,2008-12-03,NaN,30.0,38.0,Albury,25.7,12.9,1008.7,...,0.0,NaN,23.2,21.0,WSW,W,WSW,46.0,26.0,19.0
2,NaN,NaN,2008-12-04,NaN,16.0,45.0,Albury,28.0,9.2,1012.8,...,0.0,NaN,26.5,18.1,E,SE,NE,24.0,9.0,11.0
3,NaN,NaN,2008-12-06,NaN,23.0,55.0,Albury,29.7,14.6,1005.4,...,0.2,NaN,28.9,20.6,W,W,WNW,56.0,24.0,19.0
4,NaN,NaN,2008-12-08,NaN,19.0,48.0,Albury,26.7,7.7,1010.1,...,0.0,NaN,25.5,16.3,W,SSE,W,35.0,17.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28434,8.0,8.0,2017-05-29,NaN,34.0,59.0,Uluru,22.2,12.7,1021.7,...,0.0,NaN,21.0,13.9,ESE,E,E,37.0,13.0,19.0
28435,NaN,NaN,2017-06-06,NaN,6.0,29.0,Uluru,20.8,1.2,1025.9,...,0.0,NaN,19.9,7.9,E,ESE,E,43.0,28.0,13.0
28436,NaN,NaN,2017-06-13,NaN,32.0,57.0,Uluru,21.2,2.5,1023.0,...,0.0,NaN,20.3,10.3,E,ESE,ENE,35.0,19.0,17.0
28437,NaN,NaN,2017-06-14,NaN,28.0,60.0,Uluru,21.4,2.3,1022.8,...,0.0,NaN,20.3,8.9,SSE,SE,SE,22.0,7.0,11.0


In [88]:
# verificando valores nulos
data.isna().sum().sort_values(ascending = False)

Sunshine         67816
Evaporation      60843
Cloud3pm         57094
Cloud9am         53657
Pressure9am      14014
Pressure3pm      13981
WindDir9am       10013
WindGustDir       9330
WindGustSpeed     9270
WindDir3pm        3778
Humidity3pm       3610
Temp3pm           2726
WindSpeed3pm      2630
Humidity9am       1774
RainToday         1406
Rainfall          1406
WindSpeed9am      1348
Temp9am            904
MinTemp            637
MaxTemp            322
RainTomorrow         0
Location             0
Date                 0
dtype: int64

In [89]:
# tem 1406 nulos em RainToday. Pode afetar no treinamento. Dropando...
data.dropna(subset = ["RainToday"], inplace = True)

In [90]:
# Dropando Date e Location por serem categoricas e não contribuírem muito pra o resultado (não sem muito pré-processamento e análise geográfica e climatológica que eu não tenho conhecimento)
# Dropando Sunshine por estar mais da metade nula, então não enxergo a imputação como solução boa (Pode produzir dados irreais demais)
# Dropando WindDir(todos) por envolver análise geográfica que eu não tenho conhecimento (fora que são categóricos)
data.drop(['Date', 'Sunshine', 'Location', 'WindDir3pm', 'WindDir9am', 'WindGustDir'], axis=1, inplace=True) 

In [92]:
# checando de novo pra ver como tá a base de dados
data.isnull().sum()

Cloud3pm         56094
Cloud9am         52625
Evaporation      59694
Humidity3pm       3501
Humidity9am       1517
MaxTemp            307
MinTemp            468
Pressure3pm      13769
Pressure9am      13743
RainToday            0
RainTomorrow         0
Rainfall             0
Temp3pm           2624
Temp9am            656
WindGustSpeed     9105
WindSpeed3pm      2531
WindSpeed9am      1055
dtype: int64

In [93]:
# Agora será necessário fazer imputação dos resultados nulos

# Evaporation eu considero um dado muito importante pra dropar, apesar da quantidade de nulos. Acho que se imputar a moda fica mais próximo da realidade
# Cloud e Humidity por serem inteiras
data['Evaporation'].fillna(data['Evaporation'].mode()[0], inplace=True)
data['Cloud3pm'].fillna(data['Cloud3pm'].mode()[0], inplace=True)
data['Cloud9am'].fillna(data['Cloud9am'].mode()[0], inplace=True)
data['Humidity3pm'].fillna(data['Humidity3pm'].mode()[0], inplace=True)
data['Humidity9am'].fillna(data['Humidity9am'].mode()[0], inplace=True)

# Temperaturas e Pressão podem ser imputadas pela média por serem continuas
# Velocidades também acho adequado usar média por serem bem menos dados, então se o preenchimento(caso incorreto) não impacta tanto o resultado
data['MaxTemp'].fillna(data['MaxTemp'].mean(), inplace=True)
data['MinTemp'].fillna(data['MinTemp'].mean(), inplace=True)
data['Temp3pm'].fillna(data['Temp3pm'].mean(), inplace=True)
data['Temp9am'].fillna(data['Temp9am'].mean(), inplace=True)
data['Pressure3pm'].fillna(data['Pressure3pm'].mean(), inplace=True)
data['Pressure9am'].fillna(data['Pressure9am'].mean(), inplace=True)
data['WindGustSpeed'].fillna(data['WindGustSpeed'].mean(), inplace=True)
data['WindSpeed3pm'].fillna(data['WindSpeed3pm'].mean(), inplace=True)
data['WindSpeed9am'].fillna(data['WindSpeed9am'].mean(), inplace=True)

# pronto agora todos os valores nulos estão preenchidos 
data.isnull().sum()

#não houve necessidade de preencher a variável alvo pois ela já veio completamente preenchida


Cloud3pm         0
Cloud9am         0
Evaporation      0
Humidity3pm      0
Humidity9am      0
MaxTemp          0
MinTemp          0
Pressure3pm      0
Pressure9am      0
RainToday        0
RainTomorrow     0
Rainfall         0
Temp3pm          0
Temp9am          0
WindGustSpeed    0
WindSpeed3pm     0
WindSpeed9am     0
dtype: int64

In [94]:
print(data['RainTomorrow'].value_counts())
print(data['RainTomorrow'].value_counts()/data['RainTomorrow'].count())

No     109586
Yes     31201
Name: RainTomorrow, dtype: int64
No     0.778382
Yes    0.221618
Name: RainTomorrow, dtype: float64


In [95]:
# mudando os valores de RainTomorrow para números
data.RainTomorrow.replace(('Yes', 'No'), (1, 0), inplace=True)
data.RainToday.replace(('Yes', 'No'), (1, 0), inplace=True)

### Etapa 3: Divisão treino-teste e normalização dos dados

AQUI EU VOU USAR O MÉTODO HOLDOUT (70% treino/ 30% teste)

In [97]:
# dividindo o dataset entre amostras e labels
alldata_x = data.drop('RainTomorrow', axis=1)
alldata_y = data['RainTomorrow']
alldata_x.shape, alldata_y.shape

((140787, 16), (140787,))

In [98]:
# Agora dividir entre treino e teste
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

np.random.seed(seed_number)
X_train, X_test, y_train, y_test = train_test_split(alldata_x, alldata_y, test_size = 0.3, stratify=alldata_y, random_state = seed_number)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

"""
BASE DE TREINO:
AMOSTRA: 98550 amostras, com 16 atributos para avaliar
LABELS: 98550 labels

BASE DE TESTE:
AMOSTRA: 42237 amostras, com 16 atributos para avaliar
LABELS: 42237 labels
"""

'\nBASE DE TREINO:\nAMOSTRA: 98550 amostras, com 16 atributos para avaliar\nLABELS: 98550 labels\n\nBASE DE TESTE:\nAMOSTRA: 42237 amostras, com 16 atributos para avaliar\nLABELS: 42237 labels\n'

In [99]:
features_list = X_train.columns.to_list()

scaler = MinMaxScaler()

X_train[features_list] = scaler.fit_transform(X_train[features_list])
X_test[features_list] = scaler.transform(X_test[features_list])
X_train.describe()

,Cloud3pm,Cloud9am,Evaporation,Humidity3pm,Humidity9am,MaxTemp,MinTemp,Pressure3pm,Pressure9am,RainToday,Rainfall,Temp3pm,Temp9am,WindGustSpeed,WindSpeed3pm,WindSpeed9am
count,98550.000000,98550.000000,98550.000000,98550.000000,98550.000000,98550.000000,98550.000000,98550.000000,98550.000000,98550.000000,98550.000000,98550.000000,98550.000000,98550.000000,98550.000000,98550.000000
mean,0.687033,0.599284,0.033387,0.514905,0.688740,0.541868,0.487346,0.603909,0.603443,0.223359,0.006323,0.524713,0.509807,0.263229,0.214022,0.160760
std,0.304867,0.289046,0.022520,0.205795,0.194283,0.137565,0.150958,0.108693,0.114563,0.416499,0.023005,0.133334,0.136788,0.101760,0.100098,0.101704
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.500000,0.333333,0.027586,0.370000,0.565657,0.439072,0.379717,0.535831,0.532313,0.000000,0.000000,0.428295,0.411392,0.193798,0.149425,0.080460
50%,0.875000,0.777778,0.027586,0.520000,0.696970,0.529981,0.483491,0.603550,0.602969,0.000000,0.000000,0.517442,0.504219,0.255814,0.214151,0.149425
75%,0.875000,0.777778,0.037241,0.650000,0.838384,0.638298,0.596698,0.671010,0.673469,0.000000,0.002156,0.614341,0.605485,0.310078,0.275862,0.218391
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [100]:
model = xgb.XGBClassifier()

#treinando modelo
model.fit(X_train, y_train)


[16:57:58] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [101]:
# gerar predicao
y_pred = model.predict(X_test)

### Etapa 4: Avaliação do modelo

In [102]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.95      0.91     32877
           1       0.74      0.52      0.61      9360

    accuracy                           0.85     42237
   macro avg       0.81      0.74      0.76     42237
weighted avg       0.84      0.85      0.84     42237



In [76]:
y_train

,Cloud3pm,Cloud9am,Evaporation,Humidity3pm,Humidity9am,MaxTemp,MinTemp,Pressure3pm,Pressure9am,RainToday,RainTomorrow,Rainfall,Temp3pm,Temp9am,WindGustSpeed,WindSpeed3pm,WindSpeed9am
0,7.0,7.0,4.0,25.0,44.0,25.1,7.4,1007.8,1010.6,0,1,0.0,24.3,17.2,44.0,22.0,4.0
1,2.0,7.0,4.0,30.0,38.0,25.7,12.9,1008.7,1007.6,0,1,0.0,23.2,21.0,46.0,26.0,19.0
2,7.0,7.0,4.0,16.0,45.0,28.0,9.2,1012.8,1017.6,0,1,0.0,26.5,18.1,24.0,9.0,11.0
3,7.0,7.0,4.0,23.0,55.0,29.7,14.6,1005.4,1009.2,0,1,0.2,28.9,20.6,56.0,24.0,19.0
4,7.0,7.0,4.0,19.0,48.0,26.7,7.7,1010.1,1013.4,0,1,0.0,25.5,16.3,35.0,17.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28434,8.0,8.0,4.0,34.0,59.0,22.2,12.7,1021.7,1024.3,0,1,0.0,21.0,13.9,37.0,13.0,19.0
28435,7.0,7.0,4.0,6.0,29.0,20.8,1.2,1025.9,1029.8,0,1,0.0,19.9,7.9,43.0,28.0,13.0
28436,7.0,7.0,4.0,32.0,57.0,21.2,2.5,1023.0,1027.2,0,1,0.0,20.3,10.3,35.0,19.0,17.0
28437,7.0,7.0,4.0,28.0,60.0,21.4,2.3,1022.8,1026.9,0,1,0.0,20.3,8.9,22.0,7.0,11.0
